In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_excel("Concept_Data Resep.xlsx")
df

,id,nama_resep,bahan_resep,bahan_alergen,daerah_resep,umur_resep
0,1,Bubur Hati Ayam,"nasi, hati ayam, bayam, minyak jagung, garam, ...",NaN,Indonesia,6-8 Bulan
1,2,Bubur Hati Ayam Santan,"nasi, hati ayam, wortel, santan, minyak kelapa...",NaN,Indonesia,6-8 Bulan
2,3,Bubur Pepes Hati Ayam,"nasi, hati ayam, wortel, minyak kelapa, tomat,...",NaN,Indonesia,6-8 Bulan
3,4,Bubur Udang Tahu,"nasi, udang giling, santan, tahu, tomat, keman...","udang, tahu",Indonesia,6-8 Bulan
4,5,Bubur Udang Buncis,"nasi, udang giling, buncis, minyak jagung, jer...",udang,Indonesia,6-8 Bulan
...,...,...,...,...,...,...
201,202,Ikan Dori Saus Mayo,"ikan dori, brokoli, tomat merah, mayonnaise, l...","ikan, mayonnaise",Indonesia,6-8 Bulan
202,203,Beef Stroganoff,"daging wagyu, mentega truffle garlic, bawang b...","mayonnaise, krim",Indonesia,6-8 Bulan
203,204,Buffalo Chicken Toast,"ayam bagian dada, mayonnaise, lada hitam, hima...","roti, mayonnaise",Indonesia,6-8 Bulan
204,205,Pancake Satan Keju Madu,"bubuk pancake, santan kara, susu uht full crea...","keju, mayonnaise",Indonesia,6-8 Bulan


In [3]:
df[['bahan_resep','bahan_alergen']] = df[['bahan_resep','bahan_alergen']].replace(to_replace=r'[^\w\s]', value=' ', regex=True)
#replace NaN jadi blank
df = df.replace(np.nan, "")
df

,id,nama_resep,bahan_resep,bahan_alergen,daerah_resep,umur_resep
0,1,Bubur Hati Ayam,nasi hati ayam bayam minyak jagung garam ...,,Indonesia,6-8 Bulan
1,2,Bubur Hati Ayam Santan,nasi hati ayam wortel santan minyak kelapa...,,Indonesia,6-8 Bulan
2,3,Bubur Pepes Hati Ayam,nasi hati ayam wortel minyak kelapa tomat ...,,Indonesia,6-8 Bulan
3,4,Bubur Udang Tahu,nasi udang giling santan tahu tomat keman...,udang tahu,Indonesia,6-8 Bulan
4,5,Bubur Udang Buncis,nasi udang giling buncis minyak jagung jer...,udang,Indonesia,6-8 Bulan
...,...,...,...,...,...,...
201,202,Ikan Dori Saus Mayo,ikan dori brokoli tomat merah mayonnaise l...,ikan mayonnaise,Indonesia,6-8 Bulan
202,203,Beef Stroganoff,daging wagyu mentega truffle garlic bawang b...,mayonnaise krim,Indonesia,6-8 Bulan
203,204,Buffalo Chicken Toast,ayam bagian dada mayonnaise lada hitam hima...,roti mayonnaise,Indonesia,6-8 Bulan
204,205,Pancake Satan Keju Madu,bubuk pancake santan kara susu uht full crea...,keju mayonnaise,Indonesia,6-8 Bulan


In [4]:
data = pd.DataFrame(df['bahan_resep'].astype(str)+ ' '+ df['bahan_alergen'].astype(str),columns=['bahan_resep_dan_alergen'])
data

,bahan_resep_dan_alergen
0,nasi hati ayam bayam minyak jagung garam ...
1,nasi hati ayam wortel santan minyak kelapa...
2,nasi hati ayam wortel minyak kelapa tomat ...
3,nasi udang giling santan tahu tomat keman...
4,nasi udang giling buncis minyak jagung jer...
...,...
201,ikan dori brokoli tomat merah mayonnaise l...
202,daging wagyu mentega truffle garlic bawang b...
203,ayam bagian dada mayonnaise lada hitam hima...
204,bubuk pancake santan kara susu uht full crea...


In [5]:
user_input = "kacang"
def find_recipes (data_matrix, user_matrix) :
    #hitung similarity
    tfidf = CountVectorizer()
    data_matrix = tfidf.fit_transform(data['bahan_resep_dan_alergen'])
    user_matrix = tfidf.transform([user_input])
    similarities = cosine_similarity(user_matrix, data_matrix)
    #cari top 3 recipe
    #similar_recipe = [(index, row) for index, row in enumerate(similarities[0])]
    #recommended_recipe = data.iloc[[index for index, _ in similar_recipe]]
    top_n = 5
    top_indices = np.argsort(similarities[0])[:top_n]
    top_recipes = [(df[['nama_resep','bahan_resep','bahan_alergen']].iloc[i], data['bahan_resep_dan_alergen'].iloc[i]) for i in top_indices]

    return top_recipes

In [8]:
recommended_recipe = find_recipes(df, user_input)
print(recommended_recipe)

[(nama_resep                                         Bubur Hati Ayam
bahan_resep      nasi  hati ayam  bayam  minyak jagung  garam  ...
bahan_alergen                                                     
Name: 0, dtype: object, 'nasi  hati ayam  bayam  minyak jagung  garam  bawang putih  bawang merah  jahe '), (nama_resep             Pure Semangka
bahan_resep      semangka merah  air
bahan_alergen                       
Name: 117, dtype: object, 'semangka merah  air '), (nama_resep                                  Bubur Tenggiri Brokoli
bahan_resep      ikan tenggiri  kentang  wortel  air  brokoli  ...
bahan_alergen                                                 ikan
Name: 118, dtype: object, 'ikan tenggiri  kentang  wortel  air  brokoli  margarin ikan'), (nama_resep                                   Bubur Saring Makaroni
bahan_resep      bawang bombay  ayam cincang  makaroni  wortel ...
bahan_alergen                                         pasta tepung
Name: 119, dtype: object, 'bawan